# Linear Decision Tree

In dit notebook wordt een Linear Decision Tree model getraind op de geprepareerde dataset. Dit is een model die eerst een decision tree aanmaakt en bij de leaves gebruikt het model een lineare regressie model om de target waardes te voorspellen.

In [ ]:
# Importeer gebruikte libraries
from sklearn.linear_model import LinearRegression
from lineartree import LinearTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from baseline import calculate_baseline
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt
from tqdm import tqdm
from sklearn.tree import export_graphviz
import graphviz

In [ ]:
model_df = pd.read_pickle('data/model_df.pkl')
model_df.sample(5)

Er wordt een test-train split gemaakt voor het trainen en testen van het model.

In [ ]:
X = model_df.drop('anm_tot_fh', axis=1)
y = model_df['anm_tot_fh']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Bereken de RMSE en R2 score van modellen met verschillende max_depths om te kijken welke optimaal is.

In [ ]:
depths = range(1, 11) 

rmse = []
r2 = []
# Train LTR model met verschillende max_depths
for depth in tqdm(depths):
    regressor = LinearTreeRegressor(base_estimator=LinearRegression(), 
                                    linear_features=[0, 1],
                                    split_features=[2, 3, 4, 5, 6],
                                    max_depth=depth, 
                                    n_jobs=-1)
    regressor.fit(X_train, y_train)
    predictions = regressor.predict(X_test)
    rmse.append(sqrt(mean_squared_error(y_test, predictions)))
    r2.append(r2_score(y_test, predictions))

In [ ]:
# Two plots side by side, first one showing RMSE and second one showing R2 score
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ax1.plot(depths, rmse, marker='o', linestyle='-', color='b')
ax1.set_title('Depth vs. RMSE for Linear Decision Tree')
ax1.set_xlabel('Max Depth')
ax1.set_ylabel('RMSE')
ax1.set_xticks(depths)
ax1.grid(True)

ax2.plot(depths, r2, marker='o', linestyle='-', color='b')
ax2.set_title('Depth vs. R2 for Linear Decision Tree')
ax2.set_xlabel('Max Depth')
ax2.set_ylabel('R2')
ax2.set_xticks(depths)
ax2.grid(True)

plt.show()

Een max_depth van 2 blijkt optimaal voor dit model (te zien in plots hierboven).

In [ ]:
max_depth = 2
regressor = LinearTreeRegressor(base_estimator=LinearRegression(), 
                                linear_features=[0, 1],
                                split_features=[2, 3, 4, 5, 6],
                                max_depth=max_depth, 
                                n_jobs=-1)
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

y_pred = regressor.predict(X_test)

print("Root Mean Squared Error: ", rmse)
print("R-squared (R2) Score: ", r2)

baseline_rmse, baseline_r2 = calculate_baseline(model_df)
print('Baseline RMSE: ', baseline_rmse)
print('Baseline R2: ', baseline_r2)

## Conclusie

Het model is met een minimaal verschil beter dan de baseline (de RMSE is een heel klein beetje lager en de R2 score is een heel klein beetje hoger).